In [ ]:
from collections import namedtuple
import pickle
import re
import numpy as np
import pandas as pd

In [ ]:
ExtendedUser = namedtuple('extended_user', ['user', 'tweets'])

In [ ]:
pickle_file = '../Data/extended_users_attempt3.p'
with open(pickle_file, 'rb') as f:
    extended_users = [ExtendedUser(user, tweets) for user,tweets in pickle.load(f)]
print(f'n_users: {len(extended_users)}',
      f'n_tweets: {sum(len(eu.tweets) for eu in extended_users)}',
      f'n_unique_tweets: {len([tweet for eu in extended_users for tweet in eu.tweets if tweet.full_text[:4] != "RT @"])}',
      sep="\n")

n_users: 3895
n_tweets: 15713
n_unique_tweets: 10536


In [ ]:
# Initial actor list (screen names)
with open('../Data/actor_list.txt', 'r') as f:
    initial_actor_list = [line.rstrip('\n') for line in f]

In [ ]:
# Users already classified but not in initial actor list (display names)
with open('../Data/already_classified.txt', 'r') as f:
    already_classified = [line.rstrip('\n') for line in f]

In [ ]:
# Remove retweets
extended_users_RT_removed = [ExtendedUser(eu.user, 
                                         [tweet for tweet in eu.tweets if tweet.full_text[:4] != "RT @"]
                                         ) for eu in extended_users
                            ]

In [ ]:
# Filter (non-classified) actors based on inclusion criteria
extended_users_filtered = [
     eu for eu in extended_users_RT_removed if len(eu.tweets) >= 1
     and eu.user.followers_count >= 1000
     and eu.user.screen_name not in initial_actor_list
     and eu.user.screen_name not in already_classified
     ]
print( len(extended_users_filtered) )

1072


In [ ]:
# Dividing dataset into four datasets with actors to be categorized 
column_names = columns=['User name', 'Display name', 'Number of tweets', 'Number of followers', 'Description', 'Profile link']

users_list = [
    (
    eu.user.screen_name, 
    eu.user.name,
    len(eu.tweets),
    eu.user.followers_count,
    eu.user.description,
    'twitter.com/'+eu.user.screen_name
    ) 
for eu in extended_users_filtered]
users_df = pd.DataFrame(sorted(users_list, reverse=True, key=lambda x: x[2]), columns=column_names)

n_dfs = 4
dfs = [pd.DataFrame(columns=column_names) for i in range(n_dfs)]

i = 0
for _, user in users_df.iterrows():
    dfs[i] = dfs[i].append(user)
    i += 1
    if i == 4:
        i = 0
for i, df in enumerate(dfs):
    df.to_csv('cache/df'+str(i+1)+'.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=83a05b10-68f2-47cd-90b2-acf816447dd2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>